In [1]:
import os
import pandas as pd
import numpy as np
import json

In [2]:
read_dir='E:/文档/数据集/KDDCup/algebra_2005_2006/'

save_dir='../data/kddcup/'

record_data=pd.read_table(read_dir+'algebra_2005_2006_train.txt')

prob_count_limit=15 #每个学生做过的习题下限

In [3]:
records=record_data.loc[:,['Anon Student Id','Problem Name','Step Name','Correct First Attempt','KC(Default)']]\
    .drop_duplicates(subset=['Anon Student Id','Problem Name','Step Name'],keep='first').dropna(axis=0,how='any')

records['unique problem_step']=records['Problem Name']+'|'+records['Step Name']

In [4]:
record=records.loc[:,['Anon Student Id','unique problem_step','Correct First Attempt']]
record.columns=['user_id','item_id','score']

item_conc_data=records.loc[:,['unique problem_step','KC(Default)']].drop_duplicates(keep='first')

In [5]:
import re

def get_conc(txt):
    res=[]
    for txt_i in txt.split('~~'):
        res.extend(re.findall(".*SkillRule: (.*?); {.*",txt_i))
    return res

In [6]:
item_conc_data['KC(Default)']=item_conc_data['KC(Default)'].map(get_conc)

item_conc_data.columns=['item_id','knowledge_code']
item_conc_data=item_conc_data.set_index('item_id')
item_conc_data=item_conc_data[~item_conc_data.index.duplicated()]

In [7]:
tmp_item_unique=np.unique(record['item_id'])

In [8]:
iten_list,know_list=[],[]
for item in tmp_item_unique:
    item_knows=item_conc_data.loc[item,'knowledge_code']
    if len(item_knows)>0:
        iten_list.extend([item]*len(item_knows))
        know_list.extend(item_knows)
item_open_df=pd.DataFrame({'item_id':iten_list,'knowledge_code':know_list})

In [9]:
know_unique=np.unique(item_open_df['knowledge_code'])
item_unique=np.unique(item_open_df['item_id'])
stu_unique=np.unique(record['user_id'])

In [10]:
stu_old_new=dict(zip(stu_unique,range(1,len(stu_unique)+1)))
item_old_new=dict(zip(item_unique,range(1,len(item_unique)+1)))
know_old_new=dict(zip(know_unique,range(1,len(know_unique)+1)))

In [11]:
with open(save_dir+'dict_knowledge_code.json','w') as f:
    dict_knowledge_code = {str(k):know_old_new[k] for k in  know_old_new.keys()}
    json.dump(dict_knowledge_code,f)

In [12]:
record['user_id']=record['user_id'].map(stu_old_new)
record['item_id']=record['item_id'].map(item_old_new)

item_open_df['item_id']=item_open_df['item_id'].map(item_old_new)
item_open_df['knowledge_code']=item_open_df['knowledge_code'].map(know_old_new)

record=record.dropna(how='any')
item_open_df=item_open_df.dropna(how='any')

print('Done')

Done


In [13]:
# 统计每个学生做了多少道题
problem_counter=record.groupby(by='user_id').count()

filtered_stu_boundary=prob_count_limit #学生做的题超过15道才算数

filtered_stu_id=problem_counter[problem_counter['item_id']>filtered_stu_boundary].index.to_numpy()

print('有效的学生数：',len(filtered_stu_id))

record=record.set_index('user_id').loc[filtered_stu_id,:].reset_index()

有效的学生数： 425


In [18]:
know_unique=np.unique(item_open_df['knowledge_code'])
item_unique=np.unique(list(set(record['item_id'])&set(item_open_df.reset_index()['item_id'])))
stu_unique=np.unique(record['user_id'])


stu_old_new=dict(zip(stu_unique,range(1,len(stu_unique)+1)))
item_old_new=dict(zip(item_unique,range(1,len(item_unique)+1)))
know_old_new=dict(zip(know_unique,range(1,len(know_unique)+1)))

In [19]:
record['user_id']=record['user_id'].map(stu_old_new)
record['item_id']=record['item_id'].map(item_old_new)

item_open_df['item_id']=item_open_df['item_id'].map(item_old_new)
item_open_df['knowledge_code']=item_open_df['knowledge_code'].map(know_old_new)

print('Done')

Done


In [20]:
item_open_df=item_open_df.set_index('item_id')

item_df=pd.DataFrame(columns=['item_id','knowledge_code'],index=range(1,len(item_unique)+1))
for item in range(1,len(item_unique)+1):
    item_df.loc[item,'item_id']=item
    item_df.loc[item,'knowledge_code']=np.array(item_open_df.loc[item,['knowledge_code']]).reshape(-1).tolist()

In [21]:
item_df.to_csv(save_dir+'item.csv',index=False)
record.to_csv(save_dir+'record.csv',index=False)

In [22]:
print('学习者数：',len(stu_unique))
print('习题数：',len(item_unique))
print('知识点数：',len(know_unique))
print('记录数：',len(record))

学习者数： 425
习题数： 101744
知识点数： 16
记录数： 163410
